In [1]:
import os
import numpy as np
import fitsio
from astropy.table import Table
import matplotlib.pyplot as plt
%matplotlib inline

In [24]:
from importlib import resources
emlinesfile = resources.files('fastspecfit').joinpath('data/emlines.ecsv')
emlines = Table.read(emlinesfile)
emlines = emlines[emlines['amp'] > 0.1]
I = np.logical_or.reduce((emlines['name'] == 'lyalpha', emlines['name'] == 'mgii_2796', 
                          emlines['name'] == 'mgii_2803', emlines['isbroad'] == False))
emlines = emlines[I]
print(len(emlines))
emlines

13


name,restwave,amp,isbalmer,ishelium,isbroad,nicename,plotgroup
str15,float64,float32,bool,bool,bool,str60,str32
lyalpha,1215.67,1.0,False,False,True,Ly$\alpha$,lya
mgii_2796,2796.3511,1.0,False,False,True,"MgII-$\lambda\lambda$2796,2803",mgii_2796_2803
mgii_2803,2803.5324,1.0,False,False,True,"MgII-$\lambda\lambda$2796,2803",mgii_2796_2803
oii_3726,3727.0919,1.0,False,False,False,"[OII]-$\lambda\lambda$3726,29",oii_3726_29
oii_3729,3729.875,1.0,False,False,False,"[OII]-$\lambda\lambda$3726,29",oii_3726_29
neiii_3869,3869.8611,1.0,False,False,False,[NeIII]-$\lambda$3869+H6,neiii_3869_h6
hgamma,4341.684,1.0,True,False,False,H$\gamma$-$\lambda$4340+[OIII]-$\lambda$4363,hgamma_oiii_4363
hbeta,4862.683,1.0,True,False,False,H$\beta$-$\lambda$4861,hbeta
oiii_4959,4960.2937,1.0,False,False,False,"[OIII]-$\lambda\lambda$4959,5007",oiii_doublet


In [25]:
from fastspecfit.io import cache_templates, get_templates_filename
templatefile = get_templates_filename()
templatecache = cache_templates(read_linefluxes=True)
templatecache.keys()
#F = fitsio.FITS(templatefile)
#F

dict_keys(['imf', 'vdisp_nominal', 'templateinfo', 'templatewave', 'templateflux', 'templateflux_nomvdisp', 'templateflux_nolines', 'templateflux_nolines_nomvdisp', 'vdispflux', 'vdispwave', 'vdisp', 'vdisp_nominal_indx', 'linefluxes', 'linewaves'])

In [28]:
I = []
for emline in emlines:
    if emline['name'] == 'nev_3346':
        # [NeV] 3346 is not in the FSPS templates; they have [NeIII] 3343, which we do not; so skip it
        continue
    ii = np.argmin(np.abs(emline['restwave'] - templatecache['linewaves']))
    I.append(ii)
    print(f'{emline["name"]:10s} {emline["restwave"]:.2f} {templatecache["linewaves"][ii]:.2f}')
I = np.array(I)

lyalpha    1215.67 1215.67
mgii_2796  2796.35 2796.40
mgii_2803  2803.53 2803.58
oii_3726   3727.09 3727.12
oii_3729   3729.88 3730.12
neiii_3869 3869.86 3869.92
hgamma     4341.68 4341.75
hbeta      4862.68 4862.76
oiii_4959  4960.29 4960.37
oiii_5007  5008.24 5008.31
halpha     6564.61 6564.72
sii_6716   6718.29 6718.40
sii_6731   6732.67 6732.78


In [35]:
ff = '/global/cfs/cdirs/desicollab/users/ioannis/fastspecfit/redrock-templates/stacks/parent-BGS.fits'
tt = Table(fitsio.read(ff, 'FASTSPEC'))
mm = Table(fitsio.read(ff, 'METADATA'))

In [30]:
tt[:3]

TARGETID,SURVEY,PROGRAM,HEALPIX,Z,COEFF,RCHI2,RCHI2_CONT,RCHI2_PHOT,SNR_B,SNR_R,SNR_Z,SMOOTHCORR_B,SMOOTHCORR_R,SMOOTHCORR_Z,VDISP,VDISP_IVAR,AV,AGE,ZZSUN,LOGMSTAR,SFR,DN4000,DN4000_OBS,DN4000_IVAR,DN4000_MODEL,FLUX_SYNTH_G,FLUX_SYNTH_R,FLUX_SYNTH_Z,FLUX_SYNTH_SPECMODEL_G,FLUX_SYNTH_SPECMODEL_R,FLUX_SYNTH_SPECMODEL_Z,FLUX_SYNTH_PHOTMODEL_G,FLUX_SYNTH_PHOTMODEL_R,FLUX_SYNTH_PHOTMODEL_Z,FLUX_SYNTH_PHOTMODEL_W1,FLUX_SYNTH_PHOTMODEL_W2,FLUX_SYNTH_PHOTMODEL_W3,FLUX_SYNTH_PHOTMODEL_W4,ABSMAG10_DECAM_G,ABSMAG10_IVAR_DECAM_G,KCORR10_DECAM_G,ABSMAG10_DECAM_R,ABSMAG10_IVAR_DECAM_R,KCORR10_DECAM_R,ABSMAG10_DECAM_Z,ABSMAG10_IVAR_DECAM_Z,KCORR10_DECAM_Z,ABSMAG00_U,ABSMAG00_IVAR_U,KCORR00_U,ABSMAG00_B,ABSMAG00_IVAR_B,KCORR00_B,ABSMAG00_V,ABSMAG00_IVAR_V,KCORR00_V,ABSMAG01_SDSS_U,ABSMAG01_IVAR_SDSS_U,KCORR01_SDSS_U,ABSMAG01_SDSS_G,ABSMAG01_IVAR_SDSS_G,KCORR01_SDSS_G,ABSMAG01_SDSS_R,ABSMAG01_IVAR_SDSS_R,KCORR01_SDSS_R,ABSMAG01_SDSS_I,ABSMAG01_IVAR_SDSS_I,KCORR01_SDSS_I,ABSMAG01_SDSS_Z,ABSMAG01_IVAR_SDSS_Z,KCORR01_SDSS_Z,ABSMAG01_W1,ABSMAG01_IVAR_W1,KCORR01_W1,LOGLNU_1500,LOGLNU_2800,LOGL_1450,LOGL_1700,LOGL_3000,LOGL_5100,FLYA_1215_CONT,FOII_3727_CONT,FHBETA_CONT,FOIII_5007_CONT,FHALPHA_CONT,RCHI2_LINE,DELTA_LINECHI2,DELTA_LINENDOF,APERCORR,APERCORR_G,APERCORR_R,APERCORR_Z,NARROW_Z,NARROW_ZRMS,BROAD_Z,BROAD_ZRMS,UV_Z,UV_ZRMS,NARROW_SIGMA,NARROW_SIGMARMS,BROAD_SIGMA,BROAD_SIGMARMS,UV_SIGMA,UV_SIGMARMS,MGII_DOUBLET_RATIO,OII_DOUBLET_RATIO,SII_DOUBLET_RATIO,LYALPHA_MODELAMP,LYALPHA_AMP,LYALPHA_AMP_IVAR,LYALPHA_FLUX,LYALPHA_FLUX_IVAR,LYALPHA_BOXFLUX,LYALPHA_BOXFLUX_IVAR,LYALPHA_VSHIFT,LYALPHA_SIGMA,LYALPHA_CONT,LYALPHA_CONT_IVAR,LYALPHA_EW,LYALPHA_EW_IVAR,LYALPHA_FLUX_LIMIT,LYALPHA_EW_LIMIT,LYALPHA_CHI2,LYALPHA_NPIX,OI_1304_MODELAMP,OI_1304_AMP,OI_1304_AMP_IVAR,OI_1304_FLUX,OI_1304_FLUX_IVAR,OI_1304_BOXFLUX,OI_1304_BOXFLUX_IVAR,OI_1304_VSHIFT,OI_1304_SIGMA,OI_1304_CONT,OI_1304_CONT_IVAR,OI_1304_EW,OI_1304_EW_IVAR,OI_1304_FLUX_LIMIT,OI_1304_EW_LIMIT,OI_1304_CHI2,OI_1304_NPIX,SILIV_1396_MODELAMP,SILIV_1396_AMP,SILIV_1396_AMP_IVAR,SILIV_1396_FLUX,SILIV_1396_FLUX_IVAR,SILIV_1396_BOXFLUX,SILIV_1396_BOXFLUX_IVAR,SILIV_1396_VSHIFT,SILIV_1396_SIGMA,SILIV_1396_CONT,SILIV_1396_CONT_IVAR,SILIV_1396_EW,SILIV_1396_EW_IVAR,SILIV_1396_FLUX_LIMIT,SILIV_1396_EW_LIMIT,SILIV_1396_CHI2,SILIV_1396_NPIX,CIV_1549_MODELAMP,CIV_1549_AMP,CIV_1549_AMP_IVAR,CIV_1549_FLUX,CIV_1549_FLUX_IVAR,CIV_1549_BOXFLUX,CIV_1549_BOXFLUX_IVAR,CIV_1549_VSHIFT,CIV_1549_SIGMA,CIV_1549_CONT,CIV_1549_CONT_IVAR,CIV_1549_EW,CIV_1549_EW_IVAR,CIV_1549_FLUX_LIMIT,CIV_1549_EW_LIMIT,CIV_1549_CHI2,CIV_1549_NPIX,HEII_1640_MODELAMP,HEII_1640_AMP,HEII_1640_AMP_IVAR,HEII_1640_FLUX,HEII_1640_FLUX_IVAR,HEII_1640_BOXFLUX,HEII_1640_BOXFLUX_IVAR,HEII_1640_VSHIFT,HEII_1640_SIGMA,HEII_1640_CONT,HEII_1640_CONT_IVAR,HEII_1640_EW,HEII_1640_EW_IVAR,HEII_1640_FLUX_LIMIT,HEII_1640_EW_LIMIT,HEII_1640_CHI2,HEII_1640_NPIX,ALIII_1857_MODELAMP,ALIII_1857_AMP,ALIII_1857_AMP_IVAR,ALIII_1857_FLUX,ALIII_1857_FLUX_IVAR,ALIII_1857_BOXFLUX,ALIII_1857_BOXFLUX_IVAR,ALIII_1857_VSHIFT,ALIII_1857_SIGMA,ALIII_1857_CONT,ALIII_1857_CONT_IVAR,ALIII_1857_EW,ALIII_1857_EW_IVAR,ALIII_1857_FLUX_LIMIT,ALIII_1857_EW_LIMIT,ALIII_1857_CHI2,ALIII_1857_NPIX,SILIII_1892_MODELAMP,SILIII_1892_AMP,SILIII_1892_AMP_IVAR,SILIII_1892_FLUX,SILIII_1892_FLUX_IVAR,SILIII_1892_BOXFLUX,SILIII_1892_BOXFLUX_IVAR,SILIII_1892_VSHIFT,SILIII_1892_SIGMA,SILIII_1892_CONT,SILIII_1892_CONT_IVAR,SILIII_1892_EW,SILIII_1892_EW_IVAR,SILIII_1892_FLUX_LIMIT,SILIII_1892_EW_LIMIT,SILIII_1892_CHI2,SILIII_1892_NPIX,CIII_1908_MODELAMP,CIII_1908_AMP,CIII_1908_AMP_IVAR,CIII_1908_FLUX,CIII_1908_FLUX_IVAR,CIII_1908_BOXFLUX,CIII_1908_BOXFLUX_IVAR,CIII_1908_VSHIFT,CIII_1908_SIGMA,CIII_1908_CONT,CIII_1908_CONT_IVAR,CIII_1908_EW,CIII_1908_EW_IVAR,CIII_1908_FLUX_LIMIT,CIII_1908_EW_LIMIT,CIII_1908_CHI2,CIII_1908_NPIX,MGII_2796_MODELAMP,MGII_2796_AMP,MGII_2796_AMP_IVAR,MGII_2796_FLUX,MGII_2796_FLUX_IVAR,MGII_2796_BOXFLUX,MGII_2796_BOXFLUX_IVAR,MGII_2796_VSHIFT,MGII_2796_SIGMA,MGII_2796_CONT,MG

In [40]:
linefluxcols = [f'{emline.upper()}_FLUX' for emline in emlines['name']]
lineivarcols = [f'{emline.upper()}_FLUX_IVAR' for emline in emlines['name']]
allcols = ['SURVEY', 'PROGRAM', 'HEALPIX', 'TARGETID'] + linefluxcols
jj = tt[allcols]
for fluxcol, ivarcol in zip(linefluxcols, lineivarcols):
    L = tt[fluxcol] * np.sqrt(tt[ivarcol]) < 5.
    if np.any(L):
        jj[fluxcol][L] = 0.
jj

SURVEY,PROGRAM,HEALPIX,TARGETID,LYALPHA_FLUX,MGII_2796_FLUX,MGII_2803_FLUX,OII_3726_FLUX,OII_3729_FLUX,NEIII_3869_FLUX,HGAMMA_FLUX,HBETA_FLUX,OIII_4959_FLUX,OIII_5007_FLUX,HALPHA_FLUX,SII_6716_FLUX,SII_6731_FLUX
str3,str6,int32,int64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
sv3,bright,25915,39627733927462296,0.0,0.0,0.0,27.292267,34.614563,0.0,0.0,25.903568,0.0,19.48399,74.561516,17.749792,11.968279
sv3,bright,25914,39627733927462346,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.83935,9.967945,0.0
sv3,bright,25915,39627733927463205,0.0,0.0,0.0,23.896305,30.457218,0.0,0.0,24.036009,0.0,18.146423,92.442085,19.757685,13.472651
sv3,bright,25915,39627733927463584,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sv3,bright,25915,39627733927463809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.641716,0.0,0.0
sv3,bright,25915,39627733927464298,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.87249,0.0,0.0
sv3,bright,25915,39627733927464616,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27.120384,15.267055,10.992665
sv3,bright,25915,39627733927464793,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sv3,bright,25915,39627733927464814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.981492,0.0,0.0,0.0,0.0,0.0
